<a href="https://colab.research.google.com/github/Busettyvivek/Machine-Learning-Projects/blob/main/Fake_news_prediction_using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing dependencies

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords # removes unnecessary words
from nltk.stem.porter import PorterStemmer # gives the root word
from sklearn.feature_extraction.text import TfidfVectorizer # text--> int
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # checking for accuracy score of our model

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

These words doesnot make sense while training our model with the help of these nltk library we can remove stopwords

In [19]:
# Data Preprocessing
df=pd.read_csv('train.csv')

In [20]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [21]:
df.shape

(20800, 5)

In [23]:
df.isna().sum() # checking for null values in each columns

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [24]:
# replacing null values with empty strings
df=df.fillna('')

In [25]:
# check for null values after filling with empty string
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [32]:
# combining title column and author column
df['content']=df['author']+' '+df['title']

In [33]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...","Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...",Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.comWhy the Truth Might Get You ...,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss15 Civilians Killed In Single U...,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard PortnoyIranian woman jailed for fiction...,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


In [34]:
# seperating data and label columns
x=df.drop(columns='label',axis=1) #axis=1 implies removing a column for row=0
y=df['label']

In [35]:
x.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...","Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...",Ever get the feeling your life circles the rou...,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.comWhy the Truth Might Get You ...,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss15 Civilians Killed In Single U...,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard PortnoyIranian woman jailed for fiction...,Print \nAn Iranian woman has been sentenced to...,Howard PortnoyIranian woman jailed for fiction...


In [36]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [37]:
# stemming (word ---> rootword)
port_stem=PorterStemmer()


In [38]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content) # it basically takes only alphabets(lower,upper) and excludes other special characters and symbols
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[port_stem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content =" ".join(stem_content)
  return stem_content

In [39]:
df['content']=df['content'].apply(stemming)

In [40]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucushous dem aid even see comey letter...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...","Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...",Ever get the feeling your life circles the rou...,0,daniel j flynnflynn hillari clinton big woman ...
2,2,Why the Truth Might Get You Fired,Consortiumnews.comWhy the Truth Might Get You ...,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew comwhi truth might get fire trut...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss15 Civilians Killed In Single U...,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard PortnoyIranian woman jailed for fiction...,Print \nAn Iranian woman has been sentenced to...,1,howard portnoyiranian woman jail fiction unpub...


In [41]:
df['content'].head()

0    darrel lucushous dem aid even see comey letter...
1    daniel j flynnflynn hillari clinton big woman ...
2    consortiumnew comwhi truth might get fire trut...
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoyiranian woman jail fiction unpub...
Name: content, dtype: object

In [43]:
# seperating data and labels
x=df['content'].values
y=df['label'].values

In [44]:
x

array(['darrel lucushous dem aid even see comey letter jason chaffetz tweet hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynnflynn hillari clinton big woman campu breitbart flynn hillari clinton big woman campu breitbart',
       'consortiumnew comwhi truth might get fire truth might get fire',
       ...,
       'michael j de la merc rachel abramsmaci said receiv takeov approach hudson bay new york time maci said receiv takeov approach hudson bay new york time',
       'alex ansarynato russia hold parallel exercis balkan nato russia hold parallel exercis balkan',
       'david swansonwhat keep f aliv keep f aliv'], dtype=object)

In [45]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [46]:
# converting text--> num for that use Tfidfvectorizer
vectorizer=TfidfVectorizer()
vectorizer.fit(x)

TfidfVectorizer()

In [47]:
x=vectorizer.transform(x)

In [49]:
print(x)

  (0, 26880)	0.3220390383704617
  (0, 23195)	0.2900884870248826
  (0, 15319)	0.2465376110098596
  (0, 14850)	0.3302632596995315
  (0, 13042)	0.2802103018889414
  (0, 11817)	0.12394209749548626
  (0, 8195)	0.263777440740254
  (0, 6410)	0.30585245796811605
  (0, 6102)	0.20343976287599716
  (0, 5124)	0.2790716278051692
  (0, 4310)	0.41565042039626326
  (0, 587)	0.30536406053411974
  (1, 28513)	0.4172223990072618
  (1, 11536)	0.26425698937815023
  (1, 8956)	0.30365015284269475
  (1, 8952)	0.25551717408364405
  (1, 6047)	0.18355966235892754
  (1, 4878)	0.26492256517405
  (1, 3946)	0.5310113393042862
  (1, 3350)	0.21535548345726732
  (1, 2680)	0.4078873984934702
  (2, 26767)	0.453784508806227
  (2, 16690)	0.5390531498372266
  (2, 9651)	0.37952272137361526
  (2, 8750)	0.42233078438714977
  :	:
  (20797, 22540)	0.2982496692425238
  (20797, 21207)	0.32754203085496475
  (20797, 20911)	0.148842471786157
  (20797, 17864)	0.09656914220233817
  (20797, 16644)	0.1054283403820171
  (20797, 16542)	0.19

In [51]:
# splitting our dataset to train and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
# stratify =y means the fake and real data is seggregated in equal proportions for train and test

In [52]:
# training our model
model=LogisticRegression()


In [53]:
model.fit(x_train,y_train)

LogisticRegression()

In [56]:
# accuracy on training data
x_train_pred=model.predict(x_train)
training_data_accuracy=accuracy_score(x_train_pred,y_train)
print('accuracy score of the training data :',training_data_accuracy)

accuracy score of the training data : 0.9564903846153846


In [57]:
# accuracy on testing data
x_test_pred=model.predict(x_test)
testing_data_accuracy=accuracy_score(x_test_pred,y_test)
print('accuracy score on the testing data:',testing_data_accuracy)

accuracy score on the testing data: 0.9423076923076923


In [58]:
# making a prediction
x_new=x_test[1]
x_new_pred=model.predict(x_new)
print(x_new_pred)
if x_new_pred==0:
  print('The news is real')
else:
  print('The news is fake')

[0]
The news is real


In [59]:
print(y_test[1])

0
